In [1]:
import pandas as pd
import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
import os
# plt.style.use('seaborn-colorblind')
# %matplotlib inline
from feature_cleaning import rare_values as ra

# Редкие значения и высокая кардинальность

**Определение:** *Категориальная переменная*, значения которой появляются редко.

**Примечание:** В некоторых ситуациях редкие значения, подобно выбросам, могут содержать ценную информацию в наборе данных и, следовательно, требуют особого внимания. Например, редкое значение в транзакции может указывать на мошенническую операцию.

**Определение:** Количество меток внутри категориальной переменной называется кардинальностью. Высокое количество меток в переменной известно как высокая кардинальность.

## Зачем важны редкие значения и высокая кардинальность

Редкие значения в категориальных переменных могут вызывать переобучение, особенно в методах, основанных на деревьях.

- Большое количество редко встречающихся меток добавляет шум с малым количеством информации, что может вызвать переобучение. Редкие метки могут присутствовать в обучающем наборе данных, но отсутствовать в тестовом, что также может привести к переобучению на обучающем наборе.
- Редкие метки могут появиться в тестовом наборе данных, но отсутствовать в обучающем. Таким образом, модель не будет знать, как оценивать такие метки.


## Загрузка даннных 

In [2]:
use_cols = [
    'Pclass', 'Sex', 'Age', 'Fare', 'SibSp',
    'Survived'
]


# рассмотрим Pclass & SibSp's распределения по классам
# SibSp у 3/8/5 - супер редкие
# Pclass - 3 значения, но 1/2 - гораздо более редкие
data = pd.read_csv('./data/titanic.csv', usecols=use_cols)
for i in ['Pclass','SibSp']:
    print('Variable',i,'label proportion:')
    print(data[i].value_counts()/len(data))

Variable Pclass label proportion:
3    0.551066
1    0.242424
2    0.206510
Name: Pclass, dtype: float64
Variable SibSp label proportion:
0    0.682379
1    0.234568
2    0.031425
4    0.020202
3    0.017957
8    0.007856
5    0.005612
Name: SibSp, dtype: float64


## Группировка в одну категорию

Группировка значений с редкими значениями в одну категорию rare

In [3]:
# создадим кодировщик и сгрупируем редкие значения
enc = ra.GroupingRareValues(cols=['Pclass','SibSp'],threshold=0.01).fit(data)

In [4]:
# подготовим маппинг
# SibSp - значения 5 & 8 кодируем как редкие - их меньше процента в данных
# в Pclass - ничего не меняется
print(enc.mapping)

[{'col': 'Pclass', 'mapping': 3    3
1    1
2    2
dtype: int64, 'data_type': dtype('int64')}, {'col': 'SibSp', 'mapping': 0       0
1       1
2       2
4       4
3       3
8    rare
5    rare
dtype: object, 'data_type': dtype('int64')}]


In [5]:
# сделаем трансформацию данных
data2 = enc.transform(data)

In [6]:
# посмотрим
print(data2.SibSp.value_counts())

0       608
1       209
2        28
4        18
3        16
rare     12
Name: SibSp, dtype: int64


## Mode Imputation

Заменяем редкое значение самым частотным

In [7]:
# create the encoder and fit with our data
enc = ra.ModeImputation(cols=['Pclass','SibSp'],threshold=0.01).fit(data)

In [8]:
# подготовим маппинг
# SibSp - значения 5 & 8 кодируем как редкие - их меньше процента в данных
# в Pclass - ничего не меняется
print(enc.mapping)

[{'col': 'Pclass', 'mapping': 3    3
1    1
2    2
dtype: int64, 'data_type': dtype('int64')}, {'col': 'SibSp', 'mapping': 0    0
1    1
2    2
4    4
3    3
8    0
5    0
dtype: int64, 'data_type': dtype('int64')}]


In [9]:
# сделаем трансформацию данных
data3 = enc.transform(data)

In [10]:
# посмотрим
print(data3.SibSp.value_counts())

0    620
1    209
2     28
4     18
3     16
Name: SibSp, dtype: int64
